<a href="https://colab.research.google.com/github/dinky-coder/dinky-coder/blob/main/Clean_AI_Tools_with_Dependency_Injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Decoupling Tools from Agent and Other Dependencies

In [43]:
from typing import Callable, Any, Dict, List

_TOOLS: Dict[str, Dict[str, Any]] = {}

def register_tool(
    description: str,
    tags: List[str] = None,
) -> Callable:
    """
    Decorator to register a tool function.

    Args:
        description: A brief description of the tool's purpose.
        tags: A list of tags for categorizing the tool.

    Returns:
        A decorator function that registers the tool.
    """
    def decorator(func: Callable) -> Callable:
        tool_name = func.__name__
        _TOOLS[tool_name] = {
            "function": func,
            "description": description,
            "tags": tags if tags is not None else []
        }
        return func
    return decorator

def get_tool(tool_name: str) -> Dict[str, Any]:
    """
    Retrieves a registered tool by name.

    Args:
        tool_name: The name of the tool to retrieve.

    Returns:
        A dictionary containing the tool's function, description, and tags.
    """
    return _TOOLS.get(tool_name)

def list_tools(tag_filter: str = None) -> List[str]:
    """
    Lists the names of registered tools, optionally filtered by tag.

    Args:
        tag_filter: A tag to filter tools by.

    Returns:
        A list of tool names.
    """
    if tag_filter:
        return [name for name, tool_info in _TOOLS.items() if tag_filter in tool_info.get("tags", [])]
    return list(_TOOLS.keys())

In [44]:
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(code: str) -> str:
    """Review code quality and suggest improvements."""
    # But how do we access the conversation history?
    # We can't just import the agent instance - that would create tight coupling

    return prompt_expert(
        description_of_expert="""
        Senior software architect reviewing code quality
        """,
        prompt=f"Review this code:\n{code}"
    )

In [45]:
import uuid
class ActionContext:
    def __init__(self, properties: Dict=None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self):
        return self.properties.get("memory", None)

In [50]:
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    """Review code quality and suggest improvements."""
    # Get memory to understand the code's context
    memory = action_context.get_memory()

    # Extract relevant history
    development_context = []
    memories_list = [] # Initialize memories_list to an empty list
    if memory: # Check if memory is not None
        # Only attempt to get memories if memory is not None
        memories_data = memory.get_memories()
        if memories_data: # Check if get_memories returned something
            memories_list = memories_data


    if memories_list: # Now iterate over memories_list which is guaranteed to be a list
        for mem in memories_list:
            if mem.get("type") == "user":
                development_context.append(f"User: {mem.get('content')}")
            # Hypothetical scenario where our agent includes the phrase "Here's the implementation" when it generates code
            elif mem.get("type") == "assistant" and "Here's the implementation" in mem.get("content", ""):
                development_context.append(f"Implementation Decision: {mem.get('content')}")


    # Create review prompt with full context
    review_prompt = (
        "Review this code in the context of its development history:\n\n"
        "Development History:\n"
        + '\n'.join(development_context) +
        "\n\nCurrent Implementation:\n"
        f"{code}\n\n" # Use f-string for embedding the 'code' variable
        "Analyze:\n"
        "1. Does the implementation meet all stated requirements?\n"
        "2. Are all constraints and considerations from the discussion addressed?\n"
        "3. Have any requirements or constraints been overlooked?\n"
        "4. What improvements could make the code better while staying within the discussed parameters?\n"
    )


    generate_response = action_context.get("llm")
    # Ensure generate_response is callable before calling
    if callable(generate_response):
        return generate_response(review_prompt)
    else:
        return "Error: LLM generation function not provided in action context."

Handling Session or Request-Specific Dependencies

In [51]:
@register_tool(
    description="Update code review status in project management system",
    tags=["project_management"]
)
def update_review_status(action_context: ActionContext,
                        review_id: str,
                        status: str) -> dict:
    """Update the status of a code review in the project system."""
    # Get the authentication token for this specific request
    auth_token = action_context.get("auth_token")
    if not auth_token:
        raise ValueError("Authentication token not found in context")

    # Make authenticated request
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(
        f"https://...someapi.../reviews/{review_id}/status",
        headers=headers,
        json={"status": status}
    )

    if response.status_code != 200:
        raise ValueError(f"Failed to update review status: {response.text}")

    return {"status": "updated", "review_id": review_id}

In [53]:
# Define a placeholder Agent class
class Agent:
    def run(self, user_input: str, memory=None, action_context_props=None):
        """Placeholder run method."""
        print(f"Agent run with input: {user_input}")
        # In a real scenario, this would involve tool calling, etc.
        pass

    def construct_prompt(self, action_context, goals, memory):
        """Placeholder method to construct prompt."""
        return "Placeholder prompt"

    def prompt_llm_for_action(self, action_context, prompt):
        """Placeholder method to prompt LLM."""
        # In a real scenario, this would interact with an LLM and return a response
        return "Placeholder response"

    def handle_agent_response(self, action_context, response):
        """Placeholder method to handle agent response."""
        print(f"Handling agent response: {response}")
        # In a real scenario, this would parse the response and potentially call tools
        return "Placeholder result"

    def should_terminate(self, action_context, response):
        """Placeholder method to determine if agent should terminate."""
        # In a real scenario, this would have termination logic
        return True # Terminate immediately for this placeholder


# Instantiate the placeholder agent
some_agent = Agent()

In [54]:
def run(self, user_input: str, memory=None, action_context_props=None):
    """Execute the agent loop."""
    memory = memory or Memory()

    # Create context with all necessary resources
    action_context = ActionContext({
        'memory': memory,
        'llm': self.generate_response,
        # Request-specific auth
        **action_context_props
    })

    while True:
        prompt = self.construct_prompt(action_context, self.goals, memory)
        response = self.prompt_llm_for_action(action_context, prompt)
        result = self.handle_agent_response(action_context, response)

        if self.should_terminate(action_context, response):
            break

...
# Run the agent and create custom context for the action to
# pass to tools that need it
some_agent.run("Update the project status...",
               memory=...,
               # Pass request-specific auth token
               action_context_props={"auth_token": "my_auth_token"})

Agent run with input: Update the project status...
